# **Deep Learning For Lung Cancer Prediction Based on Transcriptomic Data : MLP with Embeddings learned by a VAE and No Transfer Learning**
> Author : **Aymen MERROUCHE**. <br>
> In this notebook, we implement an **MLP with VAE computed embeddings** for our binary classification task. First we train the VAE on our training data and then we use the encoder to get latent codes as embeddings. Using these embeddings we train our MLP.

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np
from torch.distributions.normal import Normal
import warnings
warnings.filterwarnings('ignore')
from modules.MLP import *
# device, cuda if available else use cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [35]:
from utils import *
from train import *
from data_utils import *
from modules.deformable_cnn import *
from modules.focal_loss import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# **1 - Get The Data :**

In [36]:
# load hyperparametrs
# data paths args
with open('./configs/data_paths.yaml', 'r') as stream:
    data_paths_args  = yaml.load(stream,Loader=yaml.Loader)
    
    
# basic cnn args
with open('./configs/vae_mlp.yaml', 'r') as stream:
    vae_mlp_args  = yaml.load(stream,Loader=yaml.Loader)

In [4]:
%%time
# Getting the data
# dataset for lung
lung_dataset = TranscriptomicVectorsDatasetLung(data_paths_args["path_to_pan_cancer_hdf5_files"])
lung_dataloader_train, lung_dataloader_validation = get_data_loaders(lung_dataset, batch_size_train = vae_mlp_args["batch_size_ft"],\
                                                                             batch_size_validation = vae_mlp_args["batch_size_ft"])

CPU times: user 11.5 s, sys: 283 ms, total: 11.8 s
Wall time: 11.7 s


# **2 - Get The Embeddings with the VAE :**

## 1- Encoder and Decoder and VAE with linear layers :

In [5]:
class Encoder(nn.Module):
    """
    VAE Encoder with linear layers
    """
    def __init__(self, dim_input, hidden, dim_z):
        super().__init__()
        self.lin1 = nn.Linear(dim_input, hidden)
        # generate covariance matrix (hyp : diagonal)
        self.sigma = nn.Linear(hidden, dim_z)
        # genrate mean vector 
        self.mu = nn.Linear(hidden, dim_z)
    def forward(self, x):
        lin1 = F.relu(self.lin1(x))
        #we infer log(sigma**2) for stability reasons
        log_sigma_squared = self.sigma(lin1) 
        mu = self.mu(lin1)
        return mu, log_sigma_squared

In [6]:
class Decoder(nn.Module):
    """
    VAE Decoder with linear layers
    """
    def __init__(self, dim_input, hidden, dim_z):
        super().__init__()
        self.lin1 = nn.Linear(dim_z, hidden)
        self.lin2 = nn.Linear(hidden, dim_input)
    def forward(self, x):
        lin1 = F.relu(self.lin1(x))
        x = torch.sigmoid(self.lin2(lin1))
        return x  

In [7]:
class VAE(nn.Module):
    """
    VAE with linear layers based Encoder and Decoder
    """
    def __init__(self, dim_input, hidden, dim_z):
        super().__init__()
        # encoder
        self.encoder = Encoder(dim_input, hidden, dim_z).to(device)
        # decoder
        self.decoder = Decoder(dim_input, hidden, dim_z).to(device)
        
    def forward(self, x):
        mu, log_sigma_squared = self.encoder(x)
        z = self.get_codes(mu, log_sigma_squared)
        return self.decoder(z), mu, log_sigma_squared

    def get_codes(self, mu, log_sigma_squared):
        # input is log(sigma**2)
        # log(sigma**2) = 2 * log(sigma)
        # log(sigma) = (1/2)*log(sigma**2)
        # sigma = exp((1/2)*log(sigma**2))
        sigma = torch.exp((1/2) * log_sigma_squared)
        # reparametrisation trick
        eps = torch.empty_like(sigma).normal_().to(device)
        return eps.mul(sigma).add_(mu)
    def sample_data(self,z):
        return self.decoder(z)

In [8]:
def variational_lower_bound(oringinal, reconstructed, mu, log_sigma_squared):
    reconstructed = reconstructed
    oringinal = oringinal
    reconstruction_term = F.binary_cross_entropy(reconstructed, oringinal, reduction='sum')
    kl_div = - (1/2) * torch.sum(1 + log_sigma_squared - mu.pow(2) - log_sigma_squared.exp())
    return reconstruction_term + kl_div

In [85]:
# dimension of the latent variable z
dim_z = 300
# dimnesion of the generator's output/discriminator's input (an mnist image)
dim_input = len(lung_dataset[0][0])
# dimension of hidden layers
hidden = 256

In [86]:
vae = VAE(dim_input, hidden, dim_z).to(device).double()
print(vae)

VAE(
  (encoder): Encoder(
    (lin1): Linear(in_features=7509, out_features=256, bias=True)
    (sigma): Linear(in_features=256, out_features=300, bias=True)
    (mu): Linear(in_features=256, out_features=300, bias=True)
  )
  (decoder): Decoder(
    (lin1): Linear(in_features=300, out_features=256, bias=True)
    (lin2): Linear(in_features=256, out_features=7509, bias=True)
  )
)


## 2 - Training The VAE :

In [87]:
# parameters
epochs = 30

# optimisation
lr = 0.001

# optimizer
optim_vae = optim.Adam(vae.parameters())

# noise to display samples during training
noise = torch.randn(vae_mlp_args["batch_size_pt"], dim_z, device=device)

In [88]:
# Fine tuning the VAE
# per epoch logging containers
v_lower_bound = []
image_and_reconstruction = []
sampled_images = []
for epoch in range(epochs):
    print("EPOCH : ",3*epoch)
    # per gradient step logging containers
    epoch_v_lower_bound = []
    
    for i, (vectors, labels) in enumerate(lung_dataloader_train):
        
        # perform a gradient step
        
        # flatten and push to device
        vectors = vectors.to(device).double()
        
        # run vae
        reconstruction, mu, log_sigma_squared = vae(vectors)

        # loss
        loss = variational_lower_bound(vectors, reconstruction, mu, log_sigma_squared)
        
        # backpropagation
        optim_vae.zero_grad()
        loss.backward()
        optim_vae.step()
        
        # logging
        epoch_v_lower_bound.append(loss.item())
        
        # display stats and an image during training
        """if i % 300 == 0 :
            print("BATCH :",i)
            print("| v_lower_bound: ", epoch_v_lower_bound[i])"""

    # per epoch logging
    v_lower_bound.append(sum(epoch_v_lower_bound) / len(epoch_v_lower_bound))

EPOCH :  0
EPOCH :  3
EPOCH :  6
EPOCH :  9
EPOCH :  12
EPOCH :  15
EPOCH :  18
EPOCH :  21
EPOCH :  24
EPOCH :  27
EPOCH :  30
EPOCH :  33
EPOCH :  36
EPOCH :  39
EPOCH :  42
EPOCH :  45
EPOCH :  48
EPOCH :  51
EPOCH :  54
EPOCH :  57
EPOCH :  60
EPOCH :  63
EPOCH :  66
EPOCH :  69
EPOCH :  72
EPOCH :  75
EPOCH :  78
EPOCH :  81
EPOCH :  84
EPOCH :  87


# **3 - Training on the Lung Cancer Dataset :**

In [89]:
# network
net = MLP(input_size = dim_z, layers = [160, 100]).to(device).double()

# loss and optimizer  
criterion = FocalLoss().to(device)
optimizer = optim.Adam(net.parameters(), lr=vae_mlp_args['lr_pt'])

# Logging + Experiment

ignore_keys = {'no_tensorboard'}
# get hyperparameters with values in a dict
hparams = {**vae_mlp_args}
# generate a name for the experiment
expe_name = '_'.join([f"{key}={val}" for key, val in hparams.items()])
print("Experimenting with : \n \t"+expe_name)
# path where to save the model
savepath = Path('/tempory/transcriptomic_data/pre_trained_mlp_vae_checkpt.pt')
# Tensorboard summary writer
if vae_mlp_args['no_tensorboard']:
    writer = None
else:
    writer = SummaryWriter("runs/runs"+"_"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+expe_name)
# start the experiment
checkpoint = CheckpointState(net, optimizer, savepath=savepath)
fit(checkpoint, criterion, lung_dataloader_train, lung_dataloader_validation, vae_mlp_args['epochs'], writer=writer, embedding_computer=vae)
if not vae_mlp_args['no_tensorboard']:
    writer.close()

Epoch 2/50:   0%|          | 0/5 [00:00<?, ?it/s, loss=1.5734e-01]

Experimenting with : 
 	epochs=50_batch_size_pt=128_lr_pt=0.001_batch_size_ft=128_lr_ft=0.001_no_tensorboard=True
Training on GPU 

Epoch 1/50, Train Loss: 1.2786e-01, Test Loss: 0.1921
Epoch 1/50, Train Accuracy: 68.99%, Test Accuracy: 57.29%
Epoch 1/50, Train AUC: 70.23%, Test AUC: 59.54%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.92      0.61      0.73       255
      Cancer       0.12      0.48      0.19        27

    accuracy                           0.60       282
   macro avg       0.52      0.55      0.46       282
weighted avg       0.84      0.60      0.68       282



Epoch 4/50:   0%|          | 0/5 [00:00<?, ?it/s, loss=1.9606e-01]

Epoch 2/50, Train Loss: 1.0685e-01, Test Loss: 0.1831
Epoch 2/50, Train Accuracy: 70.18%, Test Accuracy: 57.79%
Epoch 2/50, Train AUC: 76.14%, Test AUC: 63.57%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.94      0.59      0.72       255
      Cancer       0.14      0.63      0.23        27

    accuracy                           0.59       282
   macro avg       0.54      0.61      0.48       282
weighted avg       0.86      0.59      0.68       282

Epoch 3/50, Train Loss: 8.5609e-02, Test Loss: 0.1711
Epoch 3/50, Train Accuracy: 74.21%, Test Accuracy: 64.26%
Epoch 3/50, Train AUC: 79.31%, Test AUC: 62.92%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.93      0.66      0.78       255
      Cancer       0.15      0.56      0.23        27

    accuracy                           0.65       282
   macro avg       0.54      0.61      0.50       282
weighted 

Epoch 6/50:   0%|          | 0/5 [00:00<?, ?it/s, loss=1.0385e-01]

...

Epoch 46/50, Train Loss: 4.7184e-02, Test Loss: 0.0797
Epoch 46/50, Train Accuracy: 88.77%, Test Accuracy: 73.06%
Epoch 46/50, Train AUC: 95.99%, Test AUC: 68.80%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.92      0.77      0.84       255
      Cancer       0.14      0.37      0.21        27

    accuracy                           0.73       282
   macro avg       0.53      0.57      0.52       282
weighted avg       0.85      0.73      0.78       282

Epoch 47/50, Train Loss: 4.1579e-02, Test Loss: 0.0735
Epoch 47/50, Train Accuracy: 89.74%, Test Accuracy: 76.66%
Epoch 47/50, Train AUC: 95.51%, Test AUC: 67.60%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.93      0.79      0.85       255
      Cancer       0.17      0.41      0.24        27

    accuracy                           0.75       282
   macro avg       0.55      0.60      0.55       282
wei

Epoch 50/50:   0%|          | 0/5 [00:00<?, ?it/s, loss=2.2972e-02]

Epoch 48/50, Train Loss: 4.4886e-02, Test Loss: 0.0729
Epoch 48/50, Train Accuracy: 86.27%, Test Accuracy: 75.64%
Epoch 48/50, Train AUC: 94.20%, Test AUC: 71.24%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.93      0.79      0.85       255
      Cancer       0.17      0.41      0.24        27

    accuracy                           0.75       282
   macro avg       0.55      0.60      0.55       282
weighted avg       0.85      0.75      0.79       282

Epoch 49/50, Train Loss: 4.5324e-02, Test Loss: 0.0755
Epoch 49/50, Train Accuracy: 88.35%, Test Accuracy: 71.51%
Epoch 49/50, Train AUC: 94.27%, Test AUC: 70.72%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.92      0.76      0.83       255
      Cancer       0.14      0.37      0.20        27

    accuracy                           0.72       282
   macro avg       0.53      0.57      0.52       282
wei

Epoch 50/50: 100%|██████████| 5/5 [00:00<00:00, 69.59it/s, loss=3.0679e-02]


Epoch 50/50, Train Loss: 3.0292e-02, Test Loss: 0.0667
Epoch 50/50, Train Accuracy: 90.24%, Test Accuracy: 74.64%
Epoch 50/50, Train AUC: 95.70%, Test AUC: 70.09%
Classification Report on Val Set : 
              precision    recall  f1-score   support

   No Cancer       0.93      0.80      0.86       255
      Cancer       0.18      0.41      0.25        27

    accuracy                           0.77       282
   macro avg       0.55      0.61      0.56       282
weighted avg       0.86      0.77      0.80       282


Finished.
Best validation AUC: 7.3261e-01
With accuracy: 0.6893028815587362
